In [1]:
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
from contextlib import redirect_stdout

%matplotlib inline
plt.rcParams["figure.figsize"] = (20,3)

from pmdarima.arima import auto_arima, ADFTest

from sklearn.metrics import mean_squared_error

import multiprocessing

In [2]:
num_cores = multiprocessing.cpu_count()
print(num_cores)

8


In [6]:
pf = pd.read_csv('C:/Users/vidis/OneDrive/Desktop/Spring23/BachelorProject/pf_filled.csv')
cf = pd.read_csv('C:/Users/vidis/OneDrive/Desktop/Spring23/BachelorProject/cf_filled.csv')

arr = pf[['Time','prod_k28']].set_index(pd.to_datetime(pf['Time']))
arr = arr.drop(['Time'],axis=1)

In [ ]:
i = 0
j = 3000

mse = []

while j <(3048):
    test_ = pd.DataFrame()
    pred_ = pd.DataFrame()
    
    train = arr[i:j]
    test = arr[j:(j+24)]
    
    arima_model = auto_arima(train, start_p=0, d=1, start_q=0, 
                        max_p = 5, max_d=5, max_q=5, start_P=0,
                        D=1, start_Q=0, m=24, seasonal=True, 
                        error_action='warn', trace=True, supress_warning=True,
                        stepwise=True, random_state=20, n_fits=50, n_jobs=1) #weird multiprocessing thing
    
    pred = arima_model.predict(n_periods=24)    
    mse.append(mean_squared_error(test['prod_k28'],pred))
    
    test_ = test_.append(test)
    pred_ = pred_.append(pd.DataFrame(pred))
    pred_ = pred_.set_index(test_.index)
    
    test_.to_csv('test_.csv', mode='a', header=False)
    pred_.to_csv('pred_.csv', mode='a', header=False)
    
    plt.plot(test_)
    plt.plot(pred_)
    plt.show()
    
    plt.plot(np.arange(0,len(mse)), mse)
    plt.show()
    
    i +=24
    j +=24

Performing stepwise search to minimize aic
 ARIMA(0,1,0)(0,1,0)[24]             : AIC=10811.379, Time=0.92 sec
